In [1]:
# Data manipulation and numerical operations
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

# Machine Learning and Model Evaluation
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score

# Custom utility functions
import sys
sys.path.append('/projectnb/peaclab-mon/boztop/resource-allocation/config')

from bu_scc_data_preprocessing import preprocess_data
from ml_model_training import train_model_per_cluster


In [2]:
# Column headers, extracted from the "man 5 accounting" command:
col_headers=["qname", "hostname", "group", "owner", "job_name", "job_number", "account", "priority", 
             "submission_time", "start_time", "end_time", "failed", "exit_status", "ru_wallclock", 
             "ru_utime", "ru_stime", "ru_maxrss", "ru_ixrss", "ru_ismrss", "ru_idrss", "ru_isrss", 
             "ru_minflt", "ru_majflt", "ru_nswap", "ru_inblock", "ru_oublock", "ru_msgsnd", 
             "ru_msgrcv", "ru_nsignals", "ru_nvcsw", "ru_nivcsw", "project", "department", "granted_pe", 
             "slots", "task_number", "cpu", "mem", "io", "category", "iow", "pe_taskid", "maxvmem", 
             "arid", "ar_submission_time"]

use_col_headers = ["group", "owner", "job_name", "account", 
             "submission_time", "start_time", "end_time", "failed", "exit_status", "ru_wallclock", 
             "ru_utime", "ru_stime", "ru_maxrss", "ru_isrss", "project", "granted_pe", 
             "slots", "cpu", "mem", "io", "category"]

In [3]:
file_path = "/projectnb/peaclab-mon/boztop/resource-allocation/datasets/bu_scc/bu.accounting.2023"
processed_df = preprocess_data(file_path, col_headers)
print("Preprocessing complete! Data is ready for analysis.")



Loading and filtering data...

Calculating additional features...

Dropping unused columns...

Parsing h_rt flags...

Parsing done!

Encoding categorical features...

Encoding done!

Preprocessing complete! Data is ready for analysis.


# Menear et al.

In [4]:
#sourced from: https://github.com/NREL/eagle-jobs/blob/master/python_scripts/EVAL_FinalModel.py
def baseline_training(train_df, test_df, train_features, target_feature, filename):
    
    train_df = train_df.dropna()
    test_df = test_df.dropna()
    
    X_train = train_df[train_features]
    X_test = test_df[train_features]
    y_train = train_df[target_feature]
    y_test = test_df[target_feature]


    params = { # From Optuna HPO
        'n_estimators': 168,
        'max_depth': 7,
        'learning_rate': 0.3968571956999504,
        'gamma': 0.640232768439118,
        'subsample': 0.747747407403972,
        'colsample_bytree': 0.6280085182287491
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    pred_vs_act_df = pd.DataFrame(columns=['runtime_pred', 'runtime_act'])

    pred_vs_act_df = pd.DataFrame({'runtime_pred': y_pred, 'runtime_act': y_test})

    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    print(f'r2: {r2:.3f}, rmse: {rmse:.0f}')

    # save the results of the baseline    
    pred_vs_act_df.to_pickle(filename)
        
        

# Execution Time Prediction Experiments


In [5]:
def create_sub_dataframes(df, selected_features, n_clusters):

    feature_data = df[selected_features].replace([np.inf, -np.inf], np.nan).fillna(df.mean(numeric_only=True))
    feature_data_scaled = feature_data.values  

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    
    df['cluster'] = kmeans.fit_predict(feature_data_scaled)
    
    cluster_centers = kmeans.cluster_centers_
    
    sub_dataframes = [df[df['cluster'] == cluster].drop(columns=['cluster']) for cluster in range(n_clusters)]
    
    return sub_dataframes, cluster_centers



In [6]:
def make_predictions(test_df, train_features, target_feature, cluster_centers, models, biases, bias_type, filename, add_bias=False):
    """
    Predicts runtime for test data, considering different bias types.

    Parameters:
        test_df (pd.DataFrame): Test dataset.
        train_features (list): Features used for training the model.
        target_feature (str): Target feature to predict.
        cluster_centers (np.ndarray): Cluster centers for assigning data points.
        models (list): List of models, one for each cluster.
        biases (list): List of dictionaries with bias values for each cluster.
        bias_type (str): Type of bias to use ('mean', 'mad', 'std_dev', 'two_sigma').
        filename (str): Path to save the resulting DataFrame.
        add_bias (bool): Whether to add bias to predictions.

    Returns:
        None
    """
    test_features = test_df[train_features].values
    distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)
    
    test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)

    pred_vs_act_df = pd.DataFrame(columns=['runtime_pred', 'runtime_act', 'original_index'])

    for cluster_id, (model, bias_dict) in enumerate(zip(models, biases)):
        cluster_data = test_df[test_df['cluster'] == cluster_id]
        if not cluster_data.empty:
            X_test = cluster_data[train_features].values
            y_test = cluster_data[target_feature].values

            if add_bias:
                # Select the appropriate bias based on the specified bias type
                bias = bias_dict.get(bias_type, 0) 
                y_pred = model.predict(X_test) + bias
            else:
                y_pred = model.predict(X_test)

            new_df = pd.DataFrame({
                'runtime_pred': y_pred,
                'runtime_act': y_test,
                'original_index': cluster_data.index
            })
            pred_vs_act_df = pd.concat([pred_vs_act_df, new_df], ignore_index=True)

    # Sort results by the original index
    pred_vs_act_df = pred_vs_act_df.sort_values(by='original_index').reset_index(drop=True)

    # Save results to a file
    pred_vs_act_df.to_pickle(filename)


In [7]:
df = processed_df.copy()
df['submission_time'] = pd.to_datetime(df['submission_time'])


In [14]:
start_dates = pd.date_range(start='2023-01-01', end='2023-06-01', freq='10D')
end_dates = start_dates + pd.DateOffset(months=6)

date_pairs = list(zip(start_dates, end_dates))


train_durations = [20, 40, 60, 80, 100, 120, 140, 160] 
# test_durations = [1, 2, 3, 4, 5, 10, 15, 20] 

bias_types = ['mean', 'mad', 'std_dev', 'two_sigma']

train_features = ['group', 'owner', 'job_name', 'department', 'granted_pe','slots']
target_feature = 'execution_time' 
req_feature = 'h_rt'




In [ ]:
import time

for start_date, end_date in date_pairs:
    print(f"\n=== Starting new workflow for Start Date: {start_date.date()} ===\n")
    start_time = time.time()  # Track start time for this configuration

    for train_days in train_durations:
        print(f"[INFO] Processing: Start Date {start_date.date()}, Train {train_days} days")

        # Train split
        train_duration = timedelta(days=train_days)
        mid_date = start_date + train_duration
        train_df = df[(df['submission_time'] >= start_date) & (df['submission_time'] < mid_date)]

        # Test split
        test_df = df[(df['submission_time'] >= mid_date) & (df['submission_time'] < end_date)]

        # CLUSTERING
        print("[INFO] Creating sub-dataframes and clustering...")
        # Optimal number for clusters is selected to be 10 for the SCC dataset
        sub_dataframes, cluster_centers = create_sub_dataframes(
            df=train_df, 
            selected_features=train_features, 
            n_clusters=10
        )
        print("[INFO] Clustering completed.")
        
        # Train plain models (XGBoost and RandomForest)
        print("[INFO] Training plain models (XGBoost and RandomForest)...")
        xgb_plain_models, plain_biases = train_model_per_cluster(
            sub_dataframes, train_features, target_feature, 'xgboost', alpha=1)
        rf_plain_models, plain_biases = train_model_per_cluster(
            sub_dataframes, train_features, target_feature, 'rf', alpha=1)
        print("[INFO] Plain models training completed.")


        # BASELINE
        print("[INFO] Running baseline training...")
        baseline_training(train_df, test_df, 
                          train_features, target_feature, 
                          f'scc_start{start_date}_train{train_days}d_baseline_execution_time_pred_vs_act_df.pkl')
        print("[INFO] Baseline training completed.")


        # Plain XGBoost Predictions
        print("[INFO] Making plain XGBoost predictions...")
        make_predictions(
            test_df, train_features, target_feature, cluster_centers, 
            xgb_plain_models, plain_biases, 'mad', 
            f'scc_start{start_date}_train{train_days}d_CP_xgboost_execution_time_pred_vs_act_df.pkl', 
            add_bias=False
        )

        # Plain RandomForest Predictions
        print("[INFO] Making plain RandomForest predictions...")
        make_predictions(
            test_df, train_features, target_feature, cluster_centers, 
            rf_plain_models, plain_biases, 'mad', 
            f'scc_start{start_date}_train{train_days}d_CP_rf_execution_time_pred_vs_act_df.pkl', 
            add_bias=False
        )

        # Bias type evaluations
        for bias_type in bias_types:
            print(f"[INFO] Evaluating with bias type: {bias_type}...")

            # Buffer XGBoost
            make_predictions(
                test_df, train_features, target_feature, cluster_centers,
                xgb_plain_models, plain_biases, bias_type,
                f'scc_start{start_date}_train{train_days}d_CB_xgboost_{bias_type}_execution_time_pred_vs_act_df.pkl', 
                add_bias=True
            )

            # Buffer RandomForest
            make_predictions(
                test_df, train_features, target_feature, cluster_centers,
                rf_plain_models, plain_biases, bias_type,
                f'scc_start{start_date}_train{train_days}d_CB_rf_{bias_type}_execution_time_pred_vs_act_df.pkl', 
                add_bias=True
            )


        # USER REQUESTED
        y_test = test_df[target_feature]
        req_test = test_df[req_feature]

        user_req_df = pd.DataFrame({'runtime_req': req_test, 'runtime_act': y_test})
        filename = f'scc_start{start_date}_train{train_days}d_user_req_execution_time_pred_vs_act_df.pkl'
        user_req_df.to_pickle(filename)


        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[INFO] Completed: Start Date {start_date.date()}, Train {train_days} days - Time Elapsed: {elapsed_time:.2f} seconds\n")

            
      
            


=== Starting new workflow for Start Date: 2023-01-01 ===

[INFO] Processing: Start Date 2023-01-01, Train 20 days
[INFO] Creating sub-dataframes and clustering...


/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[INFO] Clustering completed.
[INFO] Training plain models (XGBoost and RandomForest)...
[INFO] Plain models training completed.
[INFO] Running baseline training...
r2: -0.012, rmse: 39023
[INFO] Baseline training completed.
[INFO] Making plain XGBoost predictions...


/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


[INFO] Making plain RandomForest predictions...


...

[INFO] Evaluating with bias type: two_sigma...


/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


[INFO] Completed: Start Date 2023-01-21, Train 80 days - Time Elapsed: 1095.78 seconds

[INFO] Processing: Start Date 2023-01-21, Train 100 days
[INFO] Creating sub-dataframes and clustering...


/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[INFO] Clustering completed.
[INFO] Training plain models (XGBoost and RandomForest)...
[INFO] Plain models training completed.
[INFO] Running baseline training...


# Raw - XGBoost

In [ ]:
# Train models per clusters
xgb_models, biases = train_model_per_cluster(sub_dataframes, train_features, target_feature, 'xgboost', alpha=1)

test_features = test_df[train_features].values
distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)

# find the corresponding cluster for each test sample
test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)

pred_vs_act_df = pd.DataFrame(columns=[pred, act])

for cluster_id, (model, bias) in enumerate(zip(xgb_models, biases)):
    cluster_data = test_df[test_df['cluster'] == cluster_id]
    if not cluster_data.empty:
        X_test = cluster_data[train_features].values
        y_test = cluster_data[target_feature].values
        
        y_pred = model.predict(X_test)
        new_df = pd.DataFrame({pred: y_pred, act: y_test})
        pred_vs_act_df = pd.concat([pred_vs_act_df, new_df], ignore_index=True)


filename = f'scc_CR_xgboost_{resource}_pred_vs_act_df.pkl'
pred_vs_act_df.to_pickle(filename)



# Raw - Random Forest

In [ ]:
# Train models per clusters
rf_models, biases = train_model_per_cluster(sub_dataframes, train_features, target_feature, 'rf', alpha=1)




In [ ]:
test_features = test_df[train_features].values
distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)

# find the corresponding cluster for each test sample
test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)

pred_vs_act_df = pd.DataFrame(columns=[pred, act])

for cluster_id, (model, bias) in enumerate(zip(rf_models, biases)):
    cluster_data = test_df[test_df['cluster'] == cluster_id].dropna()
    if not cluster_data.empty:
        X_test = cluster_data[train_features].values
        y_test = cluster_data[target_feature].values
        
        y_pred = model.predict(X_test)
        new_df = pd.DataFrame({pred: y_pred, act: y_test})
        pred_vs_act_df = pd.concat([pred_vs_act_df, new_df], ignore_index=True)

        
filename = f'scc_CR_rf_{resource}_pred_vs_act_df.pkl'
pred_vs_act_df.to_pickle(filename)


# Buffer - XGBoost

In [ ]:
test_features = test_df[train_features].values
distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)

# find the corresponding cluster for each test sample
test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)

pred_vs_act_df = pd.DataFrame(columns=[pred, act])

for cluster_id, (model, bias) in enumerate(zip(xgb_models, biases)):
    cluster_data = test_df[test_df['cluster'] == cluster_id]
    if not cluster_data.empty:
        X_test = cluster_data[train_features].values
        y_test = cluster_data[target_feature].values
        
        y_pred = model.predict(X_test) + bias # Add biases
        new_df = pd.DataFrame({pred: y_pred, act: y_test})
        pred_vs_act_df = pd.concat([pred_vs_act_df, new_df], ignore_index=True)


filename = f'scc_CB_xgboost_{resource}_pred_vs_act_df.pkl'
pred_vs_act_df.to_pickle(filename)



# Buffer - Random Forest

In [ ]:
test_features = test_df[train_features].values
distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)

# find the corresponding cluster for each test sample
test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)

pred_vs_act_df = pd.DataFrame(columns=[pred, act])

for cluster_id, (model, bias) in enumerate(zip(rf_models, biases)):
    cluster_data = test_df[test_df['cluster'] == cluster_id].dropna()
    if not cluster_data.empty:
        X_test = cluster_data[train_features].values
        y_test = cluster_data[target_feature].values
        
        y_pred = model.predict(X_test) + bias # Add biases
        new_df = pd.DataFrame({pred: y_pred, act: y_test})
        pred_vs_act_df = pd.concat([pred_vs_act_df, new_df], ignore_index=True)


filename = f'scc_CB_rf_{resource}_pred_vs_act_df.pkl'
pred_vs_act_df.to_pickle(filename)


# Stratified - XGBoost

In [ ]:
# Train models per clusters
models, biases = train_model_per_cluster(sub_dataframes, train_features, target_feature, 'xgboost', alpha=1, stratification=True)

test_features = test_df[train_features].values
distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)

test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)

pred_vs_act_df = pd.DataFrame(columns=[pred, act])

for cluster_id, (model, bias) in enumerate(zip(models, biases)):
    cluster_data = test_df[test_df['cluster'] == cluster_id]
    if not cluster_data.empty:
        X_test = cluster_data[train_features].values
        y_test = cluster_data[target_feature].values
        
        y_pred = model.predict(X_test) 
        new_df = pd.DataFrame({pred: y_pred, act: y_test})
        pred_vs_act_df = pd.concat([pred_vs_act_df, new_df], ignore_index=True)



filename = f'scc_CS_xgboost_{resource}_pred_vs_act_df.pkl'
pred_vs_act_df.to_pickle(filename)



# Stratified - Random Forest

In [ ]:
# Train models per clusters
models, biases = train_model_per_cluster(sub_dataframes, train_features, target_feature, 'rf', alpha=1, stratification=True)


test_features = test_df[train_features].values
distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)

test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)

pred_vs_act_df = pd.DataFrame(columns=[pred, act])

for cluster_id, (model, bias) in enumerate(zip(models, biases)):
    cluster_data = test_df[test_df['cluster'] == cluster_id].dropna()
    if not cluster_data.empty:

        X_test = cluster_data[train_features].values
        y_test = cluster_data[target_feature].values
        
        y_pred = model.predict(X_test)
        new_df = pd.DataFrame({pred: y_pred, act: y_test})
        pred_vs_act_df = pd.concat([pred_vs_act_df, new_df], ignore_index=True)



filename = f'scc_CS_rf_{resource}_pred_vs_act_df.pkl'
pred_vs_act_df.to_pickle(filename)



# User Requested

In [ ]:
req_feature = 'slots'

y_test = test_df[target_feature]
req_test = test_df[req_feature]

user_req_df = pd.DataFrame({pred: req_test, act: y_test})
        
filename = f'scc_user_req_{resource}_pred_vs_act_df.pkl'
user_req_df.to_pickle(filename)


# Cluster Train Data

In [ ]:
sub_dataframes, cluster_centers = cluster_create_sub_dataframes(train_df, train_features, 10)


# Train ML Model per Cluster

In [ ]:
models, biases = train_rf_per_cluster(sub_dataframes, train_features, 'execution_time', 2)



# Make Predictions on the Test Data

In [ ]:
test_features = test_df[train_features].values
distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)

test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)
test_df.loc[:, 'predictions'] = np.nan

for cluster_id, (model, bias) in enumerate(zip(models, biases)):
    cluster_data = test_df[test_df['cluster'] == cluster_id]
    if not cluster_data.empty:
        X_test = cluster_data[train_features].values
        X_test = np.nan_to_num(X_test, nan=0.0, posinf=np.finfo(np.float32).max, neginf=np.finfo(np.float32).min)
        test_df.loc[cluster_data.index, 'predictions'] = model.predict(X_test).flatten() + bias

#test_df = predict_test_data(test_df, models,biases, cluster_centers, train_features,'rf')



# Evaluate Predictions

In [ ]:
def evaluate_predictions_and_plot(test_df, target_column, req_column):
    metrics_per_cluster = {
        'Cluster': [],
        'RMSE': [],
        'MAE': [],
        'R2': [],
        'Underprediction %': [],
        'Overprediction %': [],
        'Sum Predicted - Actual': [],
        'Sum Requested - Actual': []
    }
    
    sum_pred_actual = []
    sum_req_actual = []
    


    for cluster_id in zip(sorted(test_df['cluster'].unique())):
        cluster_data = test_df[test_df['cluster'] == cluster_id]
        cluster_data_sorted = cluster_data.sort_index()

        y_true = cluster_data_sorted[target_column]
        y_pred = cluster_data_sorted['predictions']
        req_test = cluster_data_sorted[req_column]
        
        job_index = cluster_data_sorted.index  # Get job index

        # Plot the true values, predicted values, and requested values on the same plot
        plt.figure(figsize=(10, 6))
        plt.plot(job_index, y_true, label='True Execution Time', color='blue', linestyle='-', marker='o')
        plt.plot(job_index, y_pred, label='Predicted Execution Time', color='green', linestyle='-', marker='x')
        plt.plot(job_index, req_test, label='Requested Execution Time', color='red', linestyle='--', marker='s')

        # Add labels and title
        plt.xlabel('Job Index')
        plt.ylabel('Execution Time')
        plt.title('True, Predicted, and Requested Execution Time vs Job Index')
        plt.legend()

        # Show the plot
        plt.tight_layout()
        plt.show()
        
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

        underprediction_count = (y_pred < y_true).sum()
        overprediction_count = (y_pred > y_true).sum()
        total_count = len(y_true)
        underprediction_percent = (underprediction_count / total_count) * 100
        overprediction_percent = (overprediction_count / total_count) * 100

        op1 = y_pred > y_true
        op2 = req_test > y_true
        sum_predicted_minus_actual = (y_pred[op1] - y_true[op1]).sum()
        sum_requested_minus_actual = (req_test[op2] - y_true[op2]).sum()

        metrics_per_cluster['Cluster'].append(cluster_id)
        metrics_per_cluster['RMSE'].append(rmse)
        metrics_per_cluster['MAE'].append(mae)
        metrics_per_cluster['R2'].append(r2)
        metrics_per_cluster['Underprediction %'].append(underprediction_percent)
        metrics_per_cluster['Overprediction %'].append(overprediction_percent)
        metrics_per_cluster['Sum Predicted - Actual'].append(sum_predicted_minus_actual)
        metrics_per_cluster['Sum Requested - Actual'].append(sum_requested_minus_actual)

        sum_pred_actual.append(sum_predicted_minus_actual)
        sum_req_actual.append(sum_requested_minus_actual)

    

    avg_rmse = np.mean(metrics_per_cluster['RMSE'])
    avg_mae = np.mean(metrics_per_cluster['MAE'])
    avg_r2 = np.mean(metrics_per_cluster['R2'])
    avg_underprediction = np.mean(metrics_per_cluster['Underprediction %'])
    avg_overprediction = np.mean(metrics_per_cluster['Overprediction %'])
    avg_sum_predicted_minus_actual = np.mean(metrics_per_cluster['Sum Predicted - Actual'])
    avg_sum_requested_minus_actual = np.mean(metrics_per_cluster['Sum Requested - Actual'])
    
    metrics_per_cluster['Cluster'].append('Average')
    metrics_per_cluster['RMSE'].append(avg_rmse)
    metrics_per_cluster['MAE'].append(avg_mae)
    metrics_per_cluster['R2'].append(avg_r2)
    metrics_per_cluster['Underprediction %'].append(avg_underprediction)
    metrics_per_cluster['Overprediction %'].append(avg_overprediction)
    metrics_per_cluster['Sum Predicted - Actual'].append(avg_sum_predicted_minus_actual)
    metrics_per_cluster['Sum Requested - Actual'].append(avg_sum_requested_minus_actual)
    

    fig, ax = plt.subplots(figsize=(10, 6))
    cluster_ids = metrics_per_cluster['Cluster'][:-1]  # Exclude 'Average'
    bar_width = 0.35
    index = np.arange(len(cluster_ids))
    bar1 = ax.bar(index, sum_pred_actual, bar_width, label='Sum Predicted - Actual', color='orange')
    bar2 = ax.bar(index + bar_width, sum_req_actual, bar_width, label='Sum Requested - Actual', color='blue')
    
    ax.set_xlabel('Cluster')
    ax.set_ylabel('Sum of Differences')
    ax.set_title('Sum Predicted - Actual vs Sum Requested - Actual')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(cluster_ids)
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    return metrics_per_cluster


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

metrics = evaluate_predictions_and_plot(test_df, 'execution_time', 'h_rt')


metrics_df = pd.DataFrame(metrics)
print(metrics_df)


In [ ]:
# Remove any rows where wallclock is 0
df = df[df['ru_wallclock'] != 0]

# Keep the successful jobs only
df = df[df['failed'] == 0]
df = df[df['exit_status'] == 0]

# Add a column for the lag time
df['lag_time'] = df['start_time'] - df['submission_time']

# Remove any rows where this new column is <= 0
df = df[df['lag_time'] > 0]

df = df.dropna()

In [ ]:
# Add additional metrics
df['execution_time'] = df['end_time'] - df['start_time']

# CPU-related
df['total_cpu_time'] = df['ru_utime'] + df['ru_stime']
df['ncpu'] = np.ceil(df['total_cpu_time'] / df['ru_wallclock'])
df['cpu_waste'] = df['slots'] - df['ncpu']

df['submission_time'] = df['submission_time'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [ ]:
# --- LET'S MAKE SOME OBSERVATIONS! ---
print(f"Total number of jobs in the dataset: {df.shape[0]}")

# The total number of project groups
print(f"Total number of different groups: {df['group'].nunique()}")

print(f"Number of unique users: {df['owner'].nunique()}")
      
      

In [ ]:
df = df.drop(['account','job_number','failed','exit_status','iow','pe_taskid','arid','ar_submission_time'], axis=1)

df.columns

In [ ]:
def parse_h_rt_flag(column):
    parsed_data = []
    for entry in column:
        # Find all parts starting with '-l'
        matches = re.findall(r'-l ([^\s-]+)', entry)
        h_rt_value = None  # Default to None if h_rt is not present
        for match in matches:
            for item in match.split(','):
                if item.startswith('h_rt='):
                    _, value = item.split('=', 1)
                    h_rt_value = int(value)  # Cast the value to int
        parsed_data.append({'h_rt': h_rt_value})
    return parsed_data

# Apply the parsing function
parsed_flags = parse_h_rt_flag(df['category'])

# Convert parsed data to a DataFrame
parsed_df = pd.DataFrame(parsed_flags)

# Merge the parsed 'h_rt' column back to the original DataFrame
df = pd.concat([df, parsed_df], axis=1)

# Default hard time limit is 12 hours
df['h_rt'] = df['h_rt'].fillna(12 * 60 * 60).astype(int)


In [ ]:
df['execution_time'] = df['execution_time'].fillna(0)


# Finding Optimal Feature for Prediction

In [ ]:
numerical_submission_features = ['slots']

categorical_submission_features = ['group', 'owner', 'job_name', 'department', 'granted_pe']

label_encoders = {col: LabelEncoder() for col in categorical_submission_features}

for col in categorical_submission_features:
    df[col] = label_encoders[col].fit_transform(df[col])
    
    

# XGBoost Experiments

In [ ]:
rmse = np.sqrt(((df['slots'] - df['ncpu']) ** 2).mean())

print(f"RMSE: {rmse}")

In [ ]:
train_features = ['group', 'owner', 'job_name', 'department', 'granted_pe','slots']

# Execution Time Prediction

In [ ]:
rmse = np.sqrt(((df['execution_time'] - df['h_rt']) ** 2).mean())

print(f"RMSE: {rmse}")

In [ ]:
train_features = ['group', 'owner', 'job_name', 'department', 'granted_pe','slots']

In [ ]:
rmse, mae, r2, y_test, y_pred, req_test = train_xgboost(df, 'BU SCC Dataset', 'execution_time', train_features , 'h_rt')

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R^2: {r2:.4f}")
